In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
path = "/content/drive/MyDrive/Colab Notebooks/Tugas Akhir"

In [41]:
import pandas as pd
import os

data = pd.read_csv(os.path.join(path, "mini_preprocessed_data.csv"), encoding="latin-1")

In [42]:
print("Shape dataset:", data.shape)
data.head()

Shape dataset: (1558, 10)


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,tulisan kamu gue bisa dipahami sama gue soalnya otak gue bukan otak otak autis kayak bikin thread gobloknya sel kulit mati lanjut,1,1,1,0,0,0,1,0,0
1,semua suku bangsa melakukan tidak terkecuali arab hijaz tahu mesir mempraktikannya nama aksen aten mesopotamia awel marduk orang ibrani yahudi ishma el iya hanan etc,0,0,0,0,0,0,0,0,0
2,kalau yang bilang jangan bawa bawa agama maksudnya jangan memanfaatkan islam mengesampingkan kristen yang,0,0,0,0,0,0,0,0,0
3,pintar itu tugas utama pemerintah coba belajar tugas utama dewan perwakilan rakyat,0,0,0,0,0,0,0,0,0
4,kata cebong dolar naik ngaruhmatamu picek #gantipresiden2019,1,1,1,0,0,0,1,0,0


#Inisialisasi Model Pra-Latih cahya/bert-base-indonesian-522M

In [43]:
!pip install transformers tensorflow

In [44]:
from transformers import BertTokenizer, TFBertModel

model_name = 'cahya/bert-base-indonesian-522M'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


#Menentukan jumlah max_length

In [45]:
# Membaca data dan menyalin ke variabel baru
tokenized_data = data.copy()

# Konversi kolom 'Tweet' ke list of string
texts = tokenized_data['Tweet'].astype(str).tolist()

# Tokenisasi per kalimat, ubah ke bentuk tokens (string token, bukan ID)
tokens_list = []
for text in texts:
    encoded = tokenizer(text)  # max_length opsional
    tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])
    tokens_list.append(tokens)

# Simpan hasil tokenisasi ke kolom baru di dataframe
tokenized_data['Tokens'] = tokens_list


tokenized_data = tokenized_data.drop(columns=['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other'])

# Contoh menampilkan 5 tweet pertama beserta tokennya
pd.set_option('display.max_colwidth', None)
tokenized_data.head()

,Tweet,Tokens
0,tulisan kamu gue bisa dipahami sama gue soalnya otak gue bukan otak otak autis kayak bikin thread gobloknya sel kulit mati lanjut,"[[CLS], tulisan, kamu, gu, ##e, bisa, dipahami, sama, gu, ##e, soal, ##nya, otak, gu, ##e, bukan, otak, otak, aut, ##is, kaya, ##k, bik, ##in, thre, ##ad, gob, ##lok, ##nya, sel, kulit, mati, lanjut, [SEP]]"
1,semua suku bangsa melakukan tidak terkecuali arab hijaz tahu mesir mempraktikannya nama aksen aten mesopotamia awel marduk orang ibrani yahudi ishma el iya hanan etc,"[[CLS], semua, suku, bangsa, melakukan, tidak, terkecuali, arab, hijaz, tahu, mesir, memp, ##rak, ##tikan, ##nya, nama, aksen, aten, mesopotamia, aw, ##el, mard, ##uk, orang, ibrani, yahudi, ish, ##ma, el, i, ##ya, hana, ##n, et, ##c, [SEP]]"
2,kalau yang bilang jangan bawa bawa agama maksudnya jangan memanfaatkan islam mengesampingkan kristen yang,"[[CLS], kalau, yang, bilang, jangan, bawa, bawa, agama, maksudnya, jangan, memanfaatkan, islam, mengesampingkan, kristen, yang, [SEP]]"
3,pintar itu tugas utama pemerintah coba belajar tugas utama dewan perwakilan rakyat,"[[CLS], pintar, itu, tugas, utama, pemerintah, coba, belajar, tugas, utama, dewan, perwakilan, rakyat, [SEP]]"
4,kata cebong dolar naik ngaruhmatamu picek #gantipresiden2019,"[[CLS], kata, ce, ##bong, dolar, naik, nga, ##ruh, ##mata, ##mu, pic, ##ek, #, ganti, ##presi, ##den, ##20, ##19, [SEP]]"


In [46]:
# Hitung panjang token per tweet
token_lengths = tokenized_data['Tokens'].apply(len)

# Statistik deskriptif
print("Statistik panjang token:")
print(token_lengths.describe())

# Panjang maksimum
max_len = token_lengths.max()
print(f"\nPanjang maksimum token: {max_len}")

# Kalau ingin lihat distribusi 95% quantile
percentile_95 = token_lengths.quantile(0.95)
print(f"Panjang token yang mencakup 95% data: {int(percentile_95)}")


Statistik panjang token:
count    1558.000000
mean       19.859435
std        10.700399
min         3.000000
25%        12.000000
50%        18.000000
75%        26.000000
max        59.000000
Name: Tokens, dtype: float64

Panjang maksimum token: 59
Panjang token yang mencakup 95% data: 40


In [47]:
# Hitung panjang token dari setiap tweet
tokenized_data['Token_Length'] = tokenized_data['Tokens'].apply(len)

# Hitung panjang token 95 persen
percentile_95 = tokenized_data['Token_Length'].quantile(0.95)

# Filter baris yang panjang tokennya di atas percentile 95%
outlier_tokens = tokenized_data[tokenized_data['Token_Length'] > percentile_95]

# Tampilkan hasil
print(f"Jumlah tweet yang panjang tokennya di atas 95% data: {len(outlier_tokens)}")
outlier_tokens[['Tweet', 'Tokens', 'Token_Length']]


Jumlah tweet yang panjang tokennya di atas 95% data: 76


,Tweet,Tokens,Token_Length
26,ÃÂ¢ÃÂ¶pkl waktu indonesia bagian barat kepala unit binaan masyarakat inspektur polisi mt hutasoit melaksanakan sambang menyampaikan pesan pesan keamanan ketertiban masyarakat pilihan gubernur pilihan kepala daerah karyawan bank rakyat indonesia ranting muara damenra silit kawan kawan desa hutanagodang kecamatan muara kabupaten tap utara,"[[CLS], [UNK], [UNK], pk, ##l, waktu, indonesia, bagian, barat, kepala, unit, binaan, masyarakat, inspektur, polisi, mt, huta, ##so, ##it, melaksanakan, samba, ##ng, menyampaikan, pesan, pesan, keamanan, ketertiban, masyarakat, pilihan, gubernur, pilihan, kepala, daerah, karyawan, bank, rakyat, indonesia, ranting, muara, dame, ##n, ##ra, sili, ##t, kawan, kawan, desa, hutan, ##ago, ##dan, ##g, kecamatan, muara, kabupaten, tap, utara, [SEP]]",57
35,ssstttmengapa ribut terusssssss lihat nanti tahun siapa akan ridai allah subhanahu wa taala kembali menjadi pemimpin negara yang tercinta sudah jangan berisik bikin budek diam,"[[CLS], ss, ##st, ##tt, ##men, ##gap, ##a, ribut, terus, ##ss, ##ss, ##ss, lihat, nanti, tahun, siapa, akan, rid, ##ai, allah, sub, ##hana, ##hu, wa, ta, ##ala, kembali, menjadi, pemimpin, negara, yang, tercinta, sudah, jangan, berisi, ##k, bik, ##in, bud, ##ek, diam, [SEP]]",42
66,wajah anak kamu kelihatan banget penuh dendam kebencian anak anak kamu makan hasil uang haram bapaknya kayak kamu banci rumpi kamu memaki gue makan tai nah kamu istri anak anak orang tua kamu makan bangkai uang haram,"[[CLS], wajah, anak, kamu, kelihatan, bang, ##et, penuh, dendam, kebencian, anak, anak, kamu, makan, hasil, uang, haram, bapaknya, kaya, ##k, kamu, banc, ##i, rump, ##i, kamu, memak, ##i, gu, ##e, makan, tai, nah, kamu, istri, anak, anak, orang, tua, kamu, makan, bangkai, uang, haram, [SEP]]",45
75,nih jilboobs pintar sepong kontol hijab sange mengemut #jilbab #cantik #coli #colmek #bokep #mesum #viral hypertext transfer protocol,"[[CLS], nih, jil, ##bo, ##ob, ##s, pintar, sep, ##ong, kont, ##ol, hija, ##b, sang, ##e, mengem, ##ut, #, jilbab, #, cantik, #, coli, #, col, ##me, ##k, #, bok, ##ep, #, mes, ##um, #, viral, hyper, ##tex, ##t, transfer, protocol, [SEP]]",41
79,when tumblr tiktok telegram are blokir by the government salah orangnya di blokir aplikasinya menteri komunikasi informasi goblok when zionis kills one palestinian dasar yahudi # boikot israel jangan beli produk kafir,"[[CLS], when, tum, ##bl, ##r, tik, ##to, ##k, telegram, are, blok, ##ir, by, the, government, salah, orangnya, di, blok, ##ir, aplikasi, ##nya, menteri, komunikasi, informasi, gob, ##lok, when, zionis, kill, ##s, one, palest, ##inian, dasar, yahudi, #, bo, ##ikot, israel, jangan, beli, produk, kafir, [SEP]]",45
...,...,...,...
1498,oke jadi barusan banget malam jumat aku workout kecil kecilan kamar tidur kan nah pas mengangkat mengangkat kaki tiba tiba aku dengar suara samar samar kayak tertawa doooong mampus itu,"[[CLS], ok, ##e, jadi, barus, ##an, bang, ##et, malam, jumat, aku, work, ##out, kecil, kecil, ##an, kamar, tidur, kan, nah, pas, mengangkat, mengangkat, kaki, tiba, tiba, aku, dengar, suara, samar, samar, kaya, ##k, tertawa, doo, ##oon, ##g, mampu, ##s, itu, [SEP]]",41
1499,sejak sih boneka menang pemilihan presiden berkat bantuan besar penguasa itu dan dalangnya merajalela menjadi penguasa batas pengawasan partai dewan perwakilan rakyat tentara polisi senator organisasi masyarakat kampus mahasiswa buruh cendikiawan media ulama si,"[[CLS], sejak, sih, boneka, menang, pemilihan, presiden, berkat, bantuan, besar, penguasa, itu, dan, dalang, ##nya, mera, ##ja, ##le, ##la, menjadi, penguasa, batas, pengawasan, partai, dewan, perwakilan, rakyat, tentara, polisi, senator, organisasi, masyarakat, kampus, mahasiswa, buruh, cend, ##iki, ##awan, media, ulama, si, [SEP]]",42
1509,orang cina bebas selama sih mukidi keahlian cebong tukang tipu cebong begitu goblok enggak ketulungan plonga plongo mengakibatkan optimisme berlebihan #2019gantipresiden #plongaplongo #dunningkruger

In [48]:
# Buat rangkuman jumlah teks untuk setiap panjang token
token_lengths = tokenized_data['Token_Length']
length_summary = token_lengths.value_counts().sort_index()

# Filter hanya panjang token yang berada di atas percentile 95
outlier_summary = length_summary[length_summary.index > percentile_95]

# Tampilkan hasil
print(outlier_summary)

Token_Length
41    11
42    10
43     6
44     4
45    11
46     3
47     2
48     5
49     1
50     4
51     2
52     7
53     3
54     1
55     1
56     2
57     2
59     1
Name: count, dtype: int64


#Tokenisasi dengan Pra-latih BERT

In [49]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import time

batch_size = 32
texts = data['Tweet'].astype(str).tolist()

bert_embeddings = []

start = time.time()
for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
    batch_texts = texts[i:i+batch_size]
    encoded = tokenizer(batch_texts, return_tensors='tf', padding='max_length', truncation=True, max_length=50)
    output = model(encoded)

    bert_embeddings.append(output.last_hidden_state.numpy())
end = time.time()
print(f"Total time: {(end - start)/60:.2f} minutes")

Embedding: 100%|██████████| 49/49 [00:21<00:00,  2.28it/s]

Total time: 0.36 minutes


In [50]:
final_embeddings = np.concatenate(bert_embeddings, axis=0)
np.save("bert_embedding.npy", final_embeddings)

In [51]:
# # salin bert_embedding.npy ke google drive
# import shutil

# shutil.copy("bert_embedding.npy", "/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/bert_embedding.npy")